
## NTRU 2015
A implementação de NTRU versão de 2015 descrita no paper "**NTRU and Lattice-Based Crypto: Past, Present, and Future**" de *Joseph H. Silverman*, Brown University apresentado no encontro The Mathematics of Post-Quantum Cryptography, DIMACS Center, Rutgers University em 12–16 de Janeiro de 2015.

## Parâmetros
Vamos usar
+ $d \ge 6$ --
  parâmetro de segurança
+ $N$ --
  um primo maior ou igual  2^d
  
+ $p$ --
  um pequeno primo  3,  5 ou  7
 
+ $q$ --
  um primo da ordem de grandeza de  $p*N$
 

 

In [1]:
d = 2

N = next_prime(1 << d)
p = 3
q = next_prime(p*N)

print d, q, p, N

2 17 3 5


### anéis de polinómios

In [2]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

### geração aleatória, arredondamento módulo $q$ e compração módulo $p$

In [3]:
def vec():
    return  [choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

# comparação módulo 'p'
#def equalp(x,y,p):
#    return  all([(a-b)%p == 0  for (a,b) in zip(x,y)] )

### classe NTRU

In [4]:
class NTRU(object):
    def __init__(self):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec()); f = 1 + p*F
        # gerar as chaves
        G = Q(vec()) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
        
    def encrypt(self,m):
        r = Q(vec()) 
        return r*self.h + Q(m)

    def decrypt(self,e):
        a = e*self.f
        return prnd(qrnd(a))


### Teste

In [5]:
# Uma instância NTRU
K = NTRU() 
# Uma mensagem aleatória
m = vec()
# Cifrar
e = K.encrypt(m)  
# Decifrar e Verificar
m == K.decrypt(e)


False

## Reticulado $L(h)$

In [6]:
# Construção da matriz geradora por blocos
import numpy as np
import sage.modules.free_module_integer as fmi
# http://doc.sagemath.org/html/en/reference/modules/sage/modules/free_module_integer.html

class Lat(NTRU):
    def __init__(self):
        super(Lat,self).__init__()
        B1 = identity_matrix(ZZ,N); Bq = q*B1; B0 = matrix(ZZ,N,N,[0]*(N^2))
        h = qrnd(self.h)
        # rodar um vetor
        H = [h]
        for k in range(N-1):
            h = [h[-1]] + h[:-1]   # shift right rotate
            H = H + [h]
        H = matrix(ZZ,N,N,H)
        #print block_matrix([[B1,H],[B0,Bq]])
        self.L = fmi.IntegerLattice(block_matrix([[Bq,B0],[H,B1]]))

In [9]:
m = vec()

count = 0
cvp_found = 0
svp_found = 0
while count < 10000:
    count += 1
    l = Lat()
    m = vec()
    e = l.encrypt(m)
    
    target = [0] * N + qrnd(Q(list(e)))
    targetNeg = [-x for x in qrnd(Q(list(e)))]
    targetNeg = [0] * N + targetNeg
    
    l1   = matrix(l.L.reduced_basis)
    t    = matrix(1,2*N,targetNeg)
    zero = matrix(2*N,1,[0]*(2*N))
    M    = matrix(1,1,[q**2])
    
    L1 = block_matrix(2,2,[[l1,zero],[t,M]])
    
    Lred = fmi.IntegerLattice(L1).reduced_basis
    
    svp = np.array(Lred[0][:-1])
    a = e*(1 + p*Q(list(svp[:N])))
    if m == prnd(qrnd(a)):
        svp_found += 1
    err1 = np.array(Lred[2*N][:-1])
    y1 = err1 + target
    if m == prnd(list(y1)[N:]):
        cvp_found += 1
print cvp_found
print svp_found

88
171
